# Functies van podiumkunstenaars

Welke rollen nemen podiumkunstenaars op in hun producties? Een eerste blik op de rollen op basis van producties sinds 2010.

Eerst een aantal libraries om de data op te halen en te verwerken.

In [23]:
import psycopg2
from configparser import ConfigParser
from pandas import DataFrame
from collections import Counter

We moeten verbinding maken met de databank.

In [16]:
cfg = ConfigParser()
cfg.read("db.cfg")
knst = psycopg2.connect(host=cfg['db']['host'], port=cfg['db']['port'],
                        database=cfg['db']['db'], user=cfg['db']['user'],
                        password=cfg['db']['pwd'])
knst.set_client_encoding('UTF-8')
cur = knst.cursor()

De SQL om de gegevens op te halen is niet zo moeilijk:

In [19]:
sql = """
SELECT pr.id, f.name_nl, pe.full_name
FROM production.productions pr
JOIN production.seasons s
ON pr.season_id = s.id
JOIN production.relationships r
ON pr.id = r.production_id
JOIN production.people pe
ON r.person_id = pe.id
JOIN production.functions f
ON r.function_id = f.id
WHERE s.start_year >= 2010
"""

Even de gegevens binnenhalen en in een pandas dataframe stoppen.

In [22]:
cur.execute(sql)
os = cur.fetchall()
df = DataFrame([o for o in os], columns=["productie_id", "functie", "persoon"])

Een kleine functie om de functies van een persoon op te halen en het totaal.

In [30]:
def get_counted_functions_for_person(persoon):
    functies = Counter(df[df["persoon"] == persoon]["functie"]).most_common()
    totaal_aantal_functies = sum([f[1] for f in functies])
    functies_percentage = [(f[0], f[1] / float(totaal_aantal_functies)) for f in functies]
    return functies_percentage, totaal_aantal_functies

Loopen doorheen alle personen.

In [38]:
lines = []
for persoon in set(df["persoon"].values):
    functies, totaal = get_counted_functions_for_person(persoon)
    for f in functies:
        lijn = [persoon, totaal, f[0], f[1]]
        lines.append(lijn)
df_functies = DataFrame(lines, columns=["persoon", "totaal aantal functies", "functie", "percentage"])

Het resultaat ziet er als volgt uit:

In [39]:
df_functies.head()

,persoon,totaal aantal functies,functie,percentage
0,Gloria Biney,2,spel,1.00
1,Tina Heylen,25,van en met,0.52
2,Tina Heylen,25,kostuumontwerp,0.20
3,Tina Heylen,25,vormgeving,0.08
4,Tina Heylen,25,spel,0.08


Als voorbeeld even de persoon met het meeste aantal functies.

In [40]:
df_functies[df_functies["totaal aantal functies"] == max(df_functies["totaal aantal functies"])]

,persoon,totaal aantal functies,functie,percentage
12129,Anne Teresa De Keersmaeker,136,choreografie,0.485294
12130,Anne Teresa De Keersmaeker,136,dans,0.191176
12131,Anne Teresa De Keersmaeker,136,lichtontwerp,0.095588
12132,Anne Teresa De Keersmaeker,136,concept,0.080882
12133,Anne Teresa De Keersmaeker,136,creatie,0.036765
12134,Anne Teresa De Keersmaeker,136,decorontwerp,0.036765
12135,Anne Teresa De Keersmaeker,136,regie,0.022059
12136,Anne Teresa De Keersmaeker,136,met,0.022059
12137,Anne Teresa De Keersmaeker,136,van,0.007353
12138,Anne Teresa De Keersmaeker,136,van en met,0.007353


Zelf verder analyseren? Download de excel file:

In [41]:
df_functies.to_excel("functies.xlsx")